# Description
Compare Whittle index and mean-field approximation using (i) generated transition probabilities and (ii) ARMMAN transition proababilities.

# Setup the environment

In [ ]:
! pip install pulp
! conda install pulp

In [ ]:
import numpy as np
import pandas as pd
import pickle
from sklearn.cluster import KMeans
import pulp
import matplotlib.pyplot as plt
import time, datetime
import multiprocessing

# Generate Data (Choice 1)

In [ ]:
def get_random_RMAB():
  # distribution of the beneficiaries over the clusters
  Ni = (np.ones(N_CLS)*(N_BEN/N_CLS)).astype(int)
  assert(Ni.sum() == N_BEN)

  P = np.random.uniform(low=0.1, high=0.9, 
                        size=(N_CLS, N_STATES, N_ACTIONS, N_STATES))
  P /= P.sum(axis=-1, keepdims=1)

  R = np.zeros((N_CLS, N_STATES, N_ACTIONS))
  R[:,:,:] = np.random.uniform(size=N_STATES).reshape(1,N_STATES,1)

  C = np.zeros((N_CLS, N_STATES, N_ACTIONS))
  cost = np.random.uniform(low=0.1, high=1, size=N_ACTIONS)
  cost[0] = 0  # the first action has cost zero
  C[:,:,:] = cost.reshape(1,1,N_ACTIONS)

  return P, R, C, Ni

# Load ARMMAN Data (Choice 2)

## Convert historical data to transition matrices

In [ ]:
"""
Converts the raw historical behavior of the beneficiaries to transition matrices.
If no data is available for a (state,action) pair, the probs have been set to 0.
"""
def convert_history_to_TP(rawData):
  months = rawData.keys()

  features = []; passiveTP = []; activeTP = [];  # keeping active and passive separate because of sparsity in active

  for month in months:
    for i in range(rawData[month]['features'].shape[0]):
      if rawData[month]['isbad'][i]:
        continue
      features.append(rawData[month]['features'][i])

      # transition probs ({NE,E},{NE,E}) for passive and active
      states = np.concatenate([rawData[month]['states_pre'][i], rawData[month]['states'][i]])
      actions = np.concatenate([np.zeros(rawData[month]['states_pre'][i].shape), rawData[month]['actions'][i]])
      assert(len(states) == len(actions)+1)

      # What to do when we don't have passive probabilities from a given state?
      # If only E, then set (NE,E) to 1? If only NE, then set (E,NE) to 1? CURRENT
      # Maybe learn one from the other and the features?
      tp_counts = np.zeros((N_ACTIONS, N_STATES, N_STATES))
      for a in range(N_ACTIONS):
        for s in range(N_STATES):
          for sp in range(N_STATES):
            for j in range(len(actions)):
              if (actions[j], states[j], states[j+1]) == (a, s, sp):
                tp_counts[a,s,sp] += 1
          # hack for ARMMAN data
          if (a == 0) and (tp_counts[a,s].sum() == 0):
            tp_counts[0,s,1-s] = 1
              
      tp_counts /= tp_counts.sum(axis=-1, keepdims=1) + (tp_counts.sum(axis=-1, keepdims=1) == 0)

      passiveTP.append(tp_counts[0]); activeTP.append(tp_counts[1])
  
  features = np.array(features); passiveTP = np.array(passiveTP); activeTP = np.array(activeTP);
  assert(features.shape[0] == activeTP.shape[0])

  rawTP = {'features': features, 'passiveTP': passiveTP, 'activeTP': activeTP}
  return rawTP

## Do clustering on passive TP (and estimate active TP)

In [ ]:
"""
Clusters the passive transition probabilities.
Used in discrete mean-field.
"""
def rawTP_to_RMAB(rawTPAll, print_variance=False):
  
  # if N_BEN is given, slect a random sample of N_BEN benficiaries
  global N_BEN
  if N_BEN is not None:
    selected_bens = np.random.randint(low=0, high=rawTPAll['passiveTP'].shape[0], size=N_BEN)
    rawTP = dict()
    for key in rawTPAll.keys():
      rawTP[key] = rawTPAll[key][selected_bens,:]
  else:
    N_BEN = rawTPAll['passiveTP'].shape[0]
    rawTP = rawTPAll

  P = np.zeros((N_CLS, N_ACTIONS, N_STATES, N_STATES))

  # get the passive transition probabilities by clustering
  pTP = rawTP['passiveTP']
  kmeans = KMeans(n_clusters=N_CLS, random_state=0).fit(pTP.reshape((pTP.shape[0], N_STATES**2)))
  P[:,0,:,:] = kmeans.cluster_centers_.reshape((N_CLS, N_STATES, N_STATES))

  # get the count of beneficiaries in each cluster
  Ni = np.zeros(N_CLS)
  for i in range(N_CLS):
    Ni[i] = (kmeans.labels_ == i).sum()
  
  # estimate the active probabilitites
  activeTP = np.zeros((N_CLS, N_STATES, N_STATES))
  activeTPcount = np.zeros(N_CLS)
  for i in range(N_CLS):
    aTP = rawTP['activeTP'][kmeans.labels_ == i,:]

    aTPcounts = (aTP.sum(axis=-1, keepdims=1) > 0.5).sum(axis=0)
    aTPcounts += (aTPcounts == 0)  # avoid division by 0
    aTP = aTP.sum(axis=0) / aTPcounts
    # there may still be no data for some clusters! Set 0.5 0.5 for them?
    aTP += 0.5*(aTP.sum(axis=-1, keepdims=1) < 0.5)
    P[i,1,:,:] = aTP

  R = np.zeros((N_CLS, N_ACTIONS, N_STATES))
  R[:,:,:] = np.array([0,1]).reshape(1,1,N_STATES)

  # Compute the variance in the assigned and actual transition probabilites
  if print_variance == True:
    pVars = np.zeros(N_CLS); aVars = np.zeros(N_CLS)
    for i in range(N_CLS):
      pTP = rawTP['passiveTP'][kmeans.labels_ == i,:]
      pVar = (pTP - P[i,0,:,:]).var(axis=0).mean()

      aTP = rawTP['activeTP'][kmeans.labels_ == i,:]
      aTP0 = aTP[aTP[:,0,:].sum(axis=-1) > 0.5, :]
      aTP1 = aTP[aTP[:,1,:].sum(axis=-1) > 0.5, :]
      aVar0 = (aTP0 - P[i,1,0,:]).var(axis=0).mean()
      if np.isnan(aVar0): aVar0 = 0
      aVar1 = (aTP1 - P[i,1,1,:]).var(axis=0).mean()
      if np.isnan(aVar1): aVar1 = 0
      aVar = (aVar0 + aVar1)/2

      print(f'Cluster {i}.\t #ben = {pTP.shape[0]},\t #ben with active data = {(aTP0.shape[0], aTP1.shape[0])}\t', end=" ")
      print(f'Variance. Passive = {pVar},\t Active = {aVar} ')

      pVars[i] = pVar; aVars[i] = aVar

    print(f'Average Variance. Passive = {pVars.mean()},\t Active = {aVars.mean()}')
    print(f'Maximum Variance. Passive = {pVars.max()},\t Active = {aVars.max()}')

  return P, R, Ni

## Load from local drive

In [ ]:
def get_ARMMAN_RMAB():
  with open(f'{FILE_PREFIX}/data/jan-march-may-data-processed.pickle', 'rb') as handle:
    rawData = pickle.load(handle)

  try:
    with open(f'{FILE_PREFIX}/data/jan-march-may-rawTP.pickle', 'rb') as handle:
      rawTP = pickle.load(handle)
  except:
    rawTP = convert_history_to_TP(rawData)
    with open(f'{FILE_PREFIX}/data/jan-march-may-rawTP.pickle', 'wb') as handle:
      pickle.dump(rawTP, handle)
  
  Pold, Rold, Ni = rawTP_to_RMAB(rawTP)

  # change notation from (cls, act, state, ...) to (cls, state, act, ...) to
  # match Jackson's AAMAS code notation.
  P = np.zeros((N_CLS, N_STATES, N_ACTIONS, N_STATES))
  R = np.zeros((N_CLS, N_STATES, N_ACTIONS))
  for i in range(N_CLS):
    for s in range(N_STATES):
      for a in range(N_ACTIONS):
        P[i,s,a,:] = Pold[i,a,s,:]
        R[i,s,a] = Rold[i,a,s]

  # Construct the cost matrix used by mult-action algorithms
  C = np.zeros((N_CLS, N_STATES, N_ACTIONS))
  assert(N_ACTIONS == 2)
  C[:,:,1] = 1
  
  return P, R, C, Ni

# Generate Adverse Data for Whittle (Choice 3)

In [ ]:
def get_adverse_RMAB_1():
  # does not work for only 1 benificiary
  Ni = np.array([N_BEN])

  P = np.zeros((N_CLS, N_STATES, N_ACTIONS, N_STATES))
  # START-1
  P[:,0,0] = np.array([0,0.1,0,0,0.9])
  P[:,0,1] = np.array([0,1,0,0,0])

  # ENGAGED-1
  P[:,1,0] = np.array([0,0,0,0,1])
  P[:,1,1] = np.array([0, 0.9, 0, 0, 0.1])

  # START-2
  P[:,2,0] = np.array([0,0,0,0.1,0.9])
  P[:,2,1] = np.array([0,0,0,1,0])

  # ENGAGED-2
  P[:,3,0] = np.array([0,0,0,0,1])
  P[:,3,1] = np.array([0,0,0,0,1])

  # DROPOUT
  P[:,4,0] = np.array([0.1, 0, 0.1, 0, 0.8])
  P[:,4,1] = np.array([0.1, 0, 0.1, 0, 0.8])

  R = np.zeros((N_CLS, N_STATES, N_ACTIONS))
  R[0,:,:] = np.array([0, 0.95, 0, 1, 0]).reshape((1,N_STATES,1))

  C = np.zeros((N_CLS, N_STATES, N_ACTIONS))
  C[:,:,:] = np.array([0,1]).reshape(1,1,N_ACTIONS)

  return P, R, C, Ni

In [ ]:
def get_adverse_RMAB_2():
  assert False, "Obsolete"
  eps = 1e-3
  # works for 2 beneficiaries
  Ni = np.array([1,1])

  P = np.zeros((N_CLS, N_STATES, N_ACTIONS, N_STATES))
  P[:,:,:] = np.array([0,0,1])
  P[0,0,1] = np.array([0,1,0])
  P[1,0,1] = np.array([0,1,0])
  P[1,1,1] = np.array([0,1,0])

  R = np.zeros((N_CLS, N_STATES, N_ACTIONS))
  R[0,:,:] = np.array([0,1,0]).reshape((1,N_STATES,1))
  R[1,:,:] = np.array([0,1-eps,0]).reshape((1,N_STATES,1))

  C = np.zeros((N_CLS, N_STATES, N_ACTIONS))
  C[:,:,:] = np.array([0,1]).reshape(1,1,N_ACTIONS)

  return P, R, C, Ni

In [ ]:
def get_adverse_RMAB():
  return get_adverse_RMAB_1()

# Whittle index
Computed for finite horizon H. Currently doing backtracking, a faster algorithm may exist.

In [ ]:
TOL = 1e-3

def q_value_finite(P, R, H):
  """
  Finds the q-value by backtracking.
  """
  q0 = R;  # notice we are in notation (s,a,s') as in the lit
  t = H
  while (t > 0):
    q1 = q0.copy()
    v = q0.max(axis=-1)
    q0 = R + P.dot(v)
    t -= 1
  
  return q0


def q_value_infinite(P, R, gamma):
  """
  Finds the q-value by iterative method.
  """
  q0 = R;  # notice we are in notation (s,a,s') as in the lit
  q1 = np.zeros(R.shape)
  while (np.linalg.norm(q1-q0) > TOL*q0.size):
    q1 = q0.copy()
    v = q0.max(axis=-1)
    q0 = R + gamma*P.dot(v)
  
  return q0


def whittle_MDP(P, R, H, s, gamma, horizon_type):
  """
  Given an MDP with transition matrices P (s,a,s) and rewards R (s,a)
  computes the Whittle index for each state. 
  H is the horizon. 
  s the state for which we are computing WI.
  Uses binary search for the index and backtracking for finding Q-values.
  """
  assert(P.shape == (N_STATES, N_ACTIONS, N_STATES))
  assert(R.shape == (N_STATES, N_ACTIONS))

  # do binary search over ld (lambda)
  ld_min = -1e5; ld_max = 1e5; ld = 0

  qDiff = 1e5
  while np.abs(qDiff) >= TOL:
    ld = (ld_min + ld_max)/2
    Rld = R.copy(); Rld[:,1] = Rld[:,1] - ld
    
    if horizon_type == "finite":
      q = q_value_finite(P, Rld, H)
    else:
      assert horizon_type == "infinite"
      q = q_value_infinite(P, Rld, gamma)

    qDiff = q[s,1] - q[s,0];
    if qDiff > TOL:
      ld_min = ld
    elif qDiff < TOL:
      ld_max = ld

  return ld


def whittle_RMAB(P, R, H, gamma, horizon_type):
  assert ((N_CLS, N_STATES, N_ACTIONS, N_STATES) == P.shape)

  WI = np.zeros((N_CLS, N_STATES))
  
  for i in range(N_CLS):
    for s in range(N_STATES):
      WI[i,s] = whittle_MDP(P[i,:], R[i,:], H, s, gamma, horizon_type)
  
  return WI


def whittle_action(WI, state, Ni):
  state_shape = state.shape
  state = state.reshape(state.size)
  action = np.zeros(state.size, dtype=int)
  wi = WI.reshape(state.size).copy()

  remaining = BUDGET
  while remaining > 0:
    i_ = np.argmax(wi)
    action[i_] += min(remaining, np.floor(state[i_]))
    remaining -= action[i_]
    wi[i_] = -np.inf
  
  return action.reshape(state_shape)


def whittle(P, R, C, H, state, Ni, gamma, horizon_type):
  assert (N_ACTIONS == 2)  # Whittle works only for two actions
  WI = whittle_RMAB(P, R, H, gamma, horizon_type)
  action = np.zeros((N_CLS, N_STATES, N_ACTIONS))
  action[:,:,1] = whittle_action(WI, state, Ni)
  action[:,:,0] = state - action[:,:,1]
  return action

In [ ]:
def whittle_finite(*args, **kwargs):
  return whittle(*args, **kwargs, horizon_type="finite")

def whittle_infinite(*args, **kwargs):
  return whittle(*args, **kwargs, horizon_type="infinite")

# Mean-Field

## LP and LP rounding
Maintain only one copy throughout all experiments, which is this.

### LP

#### PuLP - COIN

In [ ]:
def mean_field_distribution(P, R, C, B, H, state, gamma,
                            sleeping_constraint, available_arms, sleeping_weeks):
  assert( (H-1, N_CLS, N_STATES, N_ACTIONS, N_STATES) == P.shape )
  assert( (N_CLS, N_STATES, N_ACTIONS) == R.shape )
  assert( (N_CLS, N_STATES, N_ACTIONS) == C.shape )
  assert( (H,) == B.shape )
  assert( (N_CLS, N_STATES) == state.shape )

  # the LP problem
  LP = pulp.LpProblem("Mean_Field", pulp.LpMaximize)

  mu = pulp.LpVariable.dicts("mu", (range(H),range(N_CLS),range(N_STATES)),
                            0, N_BEN, pulp.LpContinuous)
  alpha = pulp.LpVariable.dicts("alpha", (range(H),range(N_CLS),range(N_STATES),range(N_ACTIONS)),
                            0, N_BEN, pulp.LpContinuous)

  # Objective
  LP += (
      pulp.lpSum(alpha[t][i][s][a]*R[i,s,a]* (gamma**t)
                for t in range(H)
                for i in range(N_CLS)
                for s in range(N_STATES)
                for a in range(N_ACTIONS))
  )

  # Constraints
  # feasibility (equality)
  for t in range(H):
    for i in range(N_CLS):
      for s in range(N_STATES):
        LP += (
            pulp.lpSum(alpha[t][i][s][a] for a in range(N_ACTIONS)) == mu[t][i][s],
            f"feasibility for time {t}, arm {i}, state {s}"
        )

  # dynamics (equality)
  for t in range(H-1):
    for i in range(N_CLS):
      for sp in range(N_STATES):
        LP += (
            mu[t+1][i][sp] == pulp.lpSum(
                alpha[t][i][s][a]*P[t,i,s,a,sp]
                for s in range(N_STATES)
                for a in range(N_ACTIONS)
            ),
            f"transition dynamics for time {t}, arm {i}, state {sp}"
        )

  # budget (inequality)
  for t in range(H):
    LP += (
        pulp.lpSum(alpha[t][i][s][a]*C[i][s][a]
                  for i in range(N_CLS)
                  for s in range(N_STATES)
                  for a in range(N_ACTIONS)) <= B[t],
        f"budget constraint for time {t}"
    )

  # sleeping constraint
  if sleeping_constraint == True:
    # implemented only for one beneficiary per cluster & two actions
    assert N_CLS == N_BEN
    assert N_ACTIONS == 2
    assert sleeping_weeks > 0
    for i in range(N_CLS):
      for t in range(H):
        LP += (
            pulp.lpSum(alpha[t+sleep][i][s][1]
                        for sleep in range(np.minimum(sleeping_weeks+1,H-t))
                        for s in range(N_STATES)) <= 1,
            f"sleeping constraint for {i} in time duration [{t}...{t+sleeping_weeks}]"
        )

  # sleeping constraint 2
  if available_arms is not None:
    # implemented only for one beneficiary per cluster & two actions
    assert N_CLS == N_BEN
    assert N_ACTIONS == 2
    for i in range(N_CLS):
      for sleep in range(np.minimum(1-int(available_arms[i]), H)):
        for s in range(N_STATES):
          LP += (
              alpha[sleep][i][s][1] == 0
          )

  # initialization (equality)
  for i in range(N_CLS):
    for s in range(N_STATES):
      LP += (
          mu[0][i][s] == state[i][s],
          f"initialization for time 0, arm {i}, state {s}"
      )



  LP.solve(pulp.PULP_CBC_CMD(msg=False))

  # compute the numpy action
  alphaNow = np.zeros((N_CLS, N_STATES, N_ACTIONS))
  for i in range(N_CLS):
    for s in range(N_STATES):
      for a in range(N_ACTIONS):
        alphaNow[i][s][a] = alpha[0][i][s][a].varValue

  # due to some reason, likely floating point approximation in the LP solver,
  # action may have small negative values, which causes runtime errors later.
  # Set them to 0.
  alphaNow = np.maximum(alphaNow, 0)

  return alphaNow

#### Gurobi

In [ ]:
def mean_field_distribution_gurobi(P, R, C, B, H, state, gamma,
                            sleeping_constraint, available_arms, sleeping_weeks):
  assert( (H-1, N_CLS, N_STATES, N_ACTIONS, N_STATES) == P.shape )
  assert( (N_CLS, N_STATES, N_ACTIONS) == R.shape )
  assert( (N_CLS, N_STATES, N_ACTIONS) == C.shape )
  assert( (H,) == B.shape )
  assert( (N_CLS, N_STATES) == state.shape )

  # the LP problem
  LP = gb.Model("Mean_Field")
  LP.modelSense = gb.GRB.MAXIMIZE

  mu = np.zeros((H, N_CLS, N_STATES), dtype=object)
  alpha = np.zeros((H, N_CLS, N_STATES, N_ACTIONS), dtype=object)

  for t in range(H):
    for i in range(N_CLS):
      for s in range(N_STATES):
        mu[t,i,s] = LP.addVar(vtype=gb.GRB.CONTINUOUS, lb=0, ub=N_BEN, name=f"mu_{t}_{i}_{s}")
        for a in range(N_ACTIONS):
          alpha[t,i,s,a] = LP.addVar(vtype=gb.GRB.CONTINUOUS, lb=0, ub=N_BEN,
                                     name=f"alpha_{t}_{i}_{s}_{a}")

  # Objective
  LP.setObjective(
      gb.quicksum(alpha[t][i][s][a]*R[i,s,a]* (gamma**t)
                for t in range(H)
                for i in range(N_CLS)
                for s in range(N_STATES)
                for a in range(N_ACTIONS))
    )

  # Constraints
  # feasibility (equality)
  for t in range(H):
    for i in range(N_CLS):
      for s in range(N_STATES):
        LP.addConstr(
            gb.quicksum(alpha[t][i][s][a] for a in range(N_ACTIONS)) == mu[t][i][s],
            f"feasibility for time {t}, arm {i}, state {s}"
        )

  # dynamics (equality)
  for t in range(H-1):
    for i in range(N_CLS):
      for sp in range(N_STATES):
        LP.addConstr(
            mu[t+1][i][sp] == gb.quicksum(
                alpha[t][i][s][a]*P[t,i,s,a,sp]
                for s in range(N_STATES)
                for a in range(N_ACTIONS)
            ),
            f"transition dynamics for time {t}, arm {i}, state {sp}"
        )

  # budget (inequality)
  for t in range(H):
    LP.addConstr(
        gb.quicksum(alpha[t][i][s][a]*C[i][s][a]
                  for i in range(N_CLS)
                  for s in range(N_STATES)
                  for a in range(N_ACTIONS)) <= B[t],
        f"budget constraint for time {t}"
    )

  # sleeping constraint
  if sleeping_constraint == True:
    # implemented only for one beneficiary per cluster & two actions
    assert N_CLS == N_BEN
    assert N_ACTIONS == 2
    assert sleeping_weeks > 0
    for i in range(N_CLS):
      for t in range(H):
        LP.addConstr(
            gb.quicksum(alpha[t+sleep][i][s][1]
                        for sleep in range(np.minimum(sleeping_weeks+1,H-t))
                        for s in range(N_STATES)) <= 1,
            f"sleeping constraint for {i} in time duration [{t}...{t+sleeping_weeks}]"
        )

  # sleeping constraint (borrowed from past time-steps)
  if available_arms is not None:
    # implemented only for one beneficiary per cluster & two actions
    assert N_CLS == N_BEN
    assert N_ACTIONS == 2
    for i in range(N_CLS):
      for sleep in range(np.minimum(1-int(available_arms[i]), H)):
        for s in range(N_STATES):
          LP.addConstr(
              alpha[sleep][i][s][1] == 0
          )

  # initialization (equality)
  for i in range(N_CLS):
    for s in range(N_STATES):
      LP.addConstr(
          mu[0][i][s] == state[i][s],
          f"initialization for time 0, arm {i}, state {s}"
      )


  LP.setParam('OutputFlag', False)
  LP.optimize()

  # compute the numpy action
  alphaNow = np.zeros((N_CLS, N_STATES, N_ACTIONS))
  for v in LP.getVars():
    if 'alpha' in v.varName:
      t = int(v.varName.split('_')[1])
      i = int(v.varName.split('_')[2])
      s = int(v.varName.split('_')[3])
      a = int(v.varName.split('_')[4])
      if t == 0:
        alphaNow[i][s][a] = v.x

  # due to some reason, likely floating point approximation in the LP solver,
  # action may have small negative values, which causes runtime errors later.
  # Set them to 0.
  alphaNow = np.maximum(alphaNow, 0)

  return alphaNow

### LP rounding to get action

In [ ]:
def mean_field_action(alphaNow, state, C, sleeping_constraint, budget):
  # compute the discrete action (intervention) out of alpha
  # there are some quick hacks below due to fractional solution & rounding

  action = alphaNow.round().astype(int)

  # too many actions for a state
  for i in range(N_CLS):
    for s in range(N_STATES):
      action[i,s,0] += state[i,s] - action[i,s].sum()
      if action[i,s,0] < 0:
        # randomly select an action and decrease the count by 1
        # print("randomly select an action and decrease the count by 1")
        a = 1 + np.random.choice(np.arange(N_ACTIONS-1), p=action[i,s,1:]/action[i,s,1:].sum())
        action[i,s,a] -= 1
        action[i,s,0] += 1

  assert((action.sum(axis=-1) == state).all() and "1")

  # too many actions for budget
  too_many_actions = 0
  while (action*C).sum() > budget + 1e-3:
    p = action.copy()
    p[:,:,0] = 0  # setting 0 probability for cost 0 action (first one)
    p = p.reshape(p.size)
    ben = np.random.choice(np.arange(p.size), p = p/p.sum())
    i, s, a = np.unravel_index(ben, (N_CLS, N_STATES, N_ACTIONS))
    assert(a > 0)
    if action[i,s,a] > 0:
      action[i,s,a] -= 1
      action[i,s,0] += 1
    too_many_actions += 1
    # print(f"too_many_actions = {too_many_actions}")

  assert((action.sum(axis=-1) == state).all() and "2")

  # too few actions (only do if no sleeping constraint)
  if sleeping_constraint == False:
    # TODO: this can mess up with the sleeping constraint, avoid for now
    too_few_action = 0
    while (action*C).sum() < budget - 1e-3:
      p = action[:,:,0].copy()
      if p.sum() < 1:
        break  # too much budget
      p = p.reshape(p.size)
      ben = np.random.choice(np.arange(p.size), p=p/p.sum())
      i, s = np.unravel_index(ben, (N_CLS, N_STATES))
      a = np.random.randint(low=1, high=N_ACTIONS)
      action[i,s,a] += 1
      action[i,s,0] -= 1
      if (action*C).sum() > budget:
        action[i,s,a] -= 1
        action[i,s,0] += 1
        break
      too_few_action += 1
      # print(f"too_few_action = {too_few_action}")

  assert((action.sum(axis=-1) == state).all() and "3")

  return action

## MF wrappers for different experiments

### MF helper functions

In [ ]:
def mean_field_action_to_raw_action(cluster_labels, current_state_raw, action):
  if cluster_labels is None:
    # did not do any clustering, only one player per cluster
    # action for the player, action for everyone in the cluster
    action_raw = action.sum(axis=1)
    # one-hot to action index
    action_raw = action_raw.argmax(axis=-1)
  else:
    action_raw = np.zeros(N_BEN, dtype=int)
    for i in range(N_CLS):
      for s in range(N_STATES):
        # beneficiaries in this cluster and state
        bens = (np.arange(N_BEN))[(cluster_labels == i) & (current_state_raw == s)]
        # actions for this cluster and state
        acts = []; [ acts.extend([a]*action[i,s,a]) for a in range(N_ACTIONS) ]
        # randomly distribute the actions among the beneficiaries
        acts = np.random.permutation(acts)
        assert(acts.shape == bens.shape)
        action_raw[bens] = acts

  return action_raw

In [ ]:
def mean_field_data_stat_to_non_stat(Pstat, H):
  P = np.zeros((H-1,) + Pstat.shape)
  P[:,:,:,:,:] = Pstat.reshape((1,) + Pstat.shape)

  B = np.zeros(H)
  B[:-1] = BUDGET

  return P, B

### MF wrapper for my implementation (used vs Whittle)



In [ ]:
def mean_field(P, R, C, H, state, Ni, gamma):
  P, B = mean_field_data_stat_to_non_stat(P, H)
  
  alphaNow = mean_field_distribution(P=P, R=R, C=C, B=B, H=H, state=state, 
                                     gamma=gamma, sleeping_constraint=False, 
                                     available_arms=None, sleeping_weeks=None)
  action = mean_field_action(alphaNow=alphaNow, state=state, C=C, 
                             sleeping_constraint=False, budget=B[0])
  return action

### MF wrapper for non-stationary

In [ ]:
def mean_field_ns_wrapper(states, P_stat, klist, timestep=0, P_ns=None,
                          sleeping_constraint=True, available_arms=None, 
                          sleeping_weeks=3, clusters=None, gamma=0.99):
  
  # Modify Global constant (as they are not defined). 
  # This function is used only in Aditya's code.
  for var in ['N_BEN', 'N_CLS', 'N_ACTIONS', 'N_STATES', 'BUDGET']:
    assert var not in globals(), f"{var} should not have been defined."
  global N_BEN, N_CLS, N_ACTIONS, N_STATES  #, BUDGET no more required
    
  print(timestep, end=" "); sys.stdout.flush()
  
  (N_BEN, N_ACTIONS, N_STATES, _) = P_stat.shape

  if clusters is None:
    N_CLS = N_BEN
    clusters = np.arange(N_CLS).astype(int)
  else:
    assert clusters.shape == (N_BEN,)
    assert sleeping_constraint == False
    assert available_arms is None
    clusters = clusters.astype(int)
    N_CLS = int(clusters.max()) + 1

  H = klist.shape[0] + 1 - timestep

  assert (N_BEN,) == states.shape
  assert (N_BEN, N_ACTIONS, N_STATES, N_STATES) == P_stat.shape
  assert timestep <= klist.shape[0]

  P = np.zeros((H-1, N_CLS, N_ACTIONS, N_STATES, N_STATES))
  R = np.zeros((N_CLS, N_STATES, N_ACTIONS))
  C = np.zeros((N_CLS, N_STATES, N_ACTIONS))

  for i in range(N_CLS):
    if P_ns is None:
      P_temp = P_stat.reshape((1, N_BEN, N_ACTIONS, N_STATES, N_STATES))
    else:
      assert (N_BEN, klist.shape[0], N_ACTIONS, N_STATES, N_STATES) == P_ns.shape
      P_temp = P_ns.swapaxes(0,1).copy()
      P_temp = P_temp[timestep:,:,:,:,:]
    P[:,i,:,:,:] = P_temp[:,clusters==i,:,:,:].mean(axis=1)
    R[:,1,:] = 1
    C[:,:,1] = 1

  P = P.swapaxes(2,3)
  assert (H-1, N_CLS, N_STATES, N_ACTIONS, N_STATES) == P.shape

  B = np.zeros(H)
  B[:-1] = klist[timestep:]

  state = np.zeros((N_CLS, N_STATES)).astype(int)
  states = states.astype(int)
  for i_ben in range(N_BEN):
    state[clusters[i_ben], states[i_ben]] += 1

  if N_STATES == 2 and N_CLS == N_BEN: assert (state[:,1] == states).all()
  
  alphaNow = mean_field_distribution(P=P, R=R, C=C, B=B, H=H, state=state, 
                                     gamma=gamma, 
                                     sleeping_constraint=sleeping_constraint, 
                                     available_arms=available_arms, 
                                     sleeping_weeks=sleeping_weeks)
  
  action = mean_field_action(alphaNow=alphaNow, state=state, C=C, 
                             sleeping_constraint=sleeping_constraint, 
                             budget=B[0])

  actions = mean_field_action_to_raw_action(cluster_labels=clusters,
                                            current_state_raw=states, 
                                            action=action)

  assert actions.sum() <= klist[timestep] + 0.1, f"Not respecting the budget {actions.sum()} vs {klist[timestep]}."
  if available_arms is not None: 
    assert (actions < np.maximum(0,available_arms) + 0.1).all(), "Not respecting availability."

  return actions

# Random
Selects the beneficiaries randomly

In [ ]:
def random(P, R, C, H, state, Ni, gamma):
  action = np.zeros((N_CLS, N_STATES, N_ACTIONS), dtype=int)
  action[:,:,0] = state  # put every action as 0

  while (action*C).sum() < BUDGET - 1e-3:
    p = action[:,:,0].reshape(N_CLS*N_STATES)
    if p.sum() < 1:
      break  # too much budget
    ben = np.random.choice(np.arange(p.size), p=p/p.sum())
    i, s = np.unravel_index(ben, (N_CLS, N_STATES))
    a = np.random.randint(low=1, high=N_ACTIONS)
    action[i,s,a] += 1
    action[i,s,0] -= 1
    if (action*C).sum() > BUDGET:
      action[i,s,a] -= 1
      action[i,s,0] += 1
      break
  
  assert((action.sum(axis=-1) == state).all())

  return action


# Run the Experiment

## helper functions

In [ ]:
def get_random_start_state(Ni):
  state = np.zeros((N_CLS, N_STATES))
  for i in range(N_CLS):
    state[i,:] = np.random.multinomial(Ni[i], pvals=np.ones(N_STATES)/N_STATES)

  return state.astype(int)

def compute_reward(alg, R, state, action):
  # the action array represents the num of active actions for each state
  assert(R.shape == action.shape)
  reward = (R*action).sum()
  return reward

def next_state(P, state, action):
  assert ((N_CLS, N_STATES, N_ACTIONS, N_STATES) == P.shape)
  assert((action.sum(axis=-1) == state).all())

  next_state = np.zeros(state.shape).astype(int)
  for i in range(N_CLS):
    for s in range(N_STATES):
      for a in range(N_ACTIONS):
        next_state[i] += np.random.multinomial(action[i,s,a], pvals=P[i,s,a,:]/P[i,s,a,:].sum())
  
  assert((state.sum(axis=-1) == next_state.sum(axis=-1)).all())
  return next_state

## save backup

In [ ]:
def save_backup():
  with open(f'{FILE_PREFIX}/data/backup/rewards_{DATA}_runs_{N_RUNS}_bens_{N_BEN}_budget_{BUDGET}_clusters_{N_CLS}_states_{N_STATES}_actions_{N_ACTIONS}_horizon_{H}_gamma_{GAMMA}.pickle', 'wb') as handle:
    pickle.dump((H_RANGE, rewards), handle)

  with open(f'{FILE_PREFIX}/data/backup/time_taken_{DATA}_runs_{N_RUNS}_bens_{N_BEN}_budget_{BUDGET}_clusters_{N_CLS}_states_{N_STATES}_actions_{N_ACTIONS}_horizon_{H}_gamma_{GAMMA}.pickle', 'wb') as handle:
    pickle.dump((calls, time_taken), handle)

## Parameters

In [ ]:
# Parameters
FILE_PREFIX = '/usr/local/google/home/abheekghosh'  # for drive it may be different
DATA = "RANDOM"

if DATA == "ARMMAN":
  N_STATES = 2
  N_ACTIONS = 2
  N_BEN = None  # if None, to be set based on the dataset, ~ 98,000
  BUDGET = 1000  # number of interventions
  N_CLS = 40  # number of clusters
  GAMMA = 0.95
elif DATA == "ADVERSE":
  N_STATES = 5
  N_ACTIONS = 2
  N_BEN = 100
  BUDGET = 50
  N_CLS = 1
  GAMMA = 0.999
elif DATA == "RANDOM":
  N_STATES = 5
  N_ACTIONS = 2
  N_BEN = 100000  # number of beneficiaries
  BUDGET = 1000  # number of interventions
  N_CLS = 40  # number of clusters
  GAMMA = 0.95
else:
  assert(False)

H_RANGE = np.concatenate([np.arange(2, 20), np.arange(20, 40, 4), np.arange(40, 80, 8), np.arange(80, 101, 10)])
# H_RANGE = np.arange(10,101,10)
# H_RANGE = np.array([5, 10])
N_RUNS = 40  # number of runs for the experiement (48 clusters in the machine)

# Algorithms
ALGOS = ['rd', 'wi-if', 'wi-f', 'mf']
ALGOS_NAMES = {'rd': 'Random', 'wi-if': 'WhittleIF', 'wi-f': 'WhittleF', 'mf': 'MeanField'}
ALGORITHMS = {'rd': random, 'wi-if': whittle_infinite, 'wi-f': whittle_finite, 'mf': mean_field}

## runs

### parallel worker

In [ ]:
def worker(P, R, C, Ni, state, rewards_mp, calls_mp, time_taken_mp):
  # rewards, calls, and time taken for this run
  reward = dict(); call = np.zeros(H); time_tk = dict()
  for alg in ALGOS: 
    reward[alg] = 0
    time_tk[alg] = np.zeros(H)

  for t in range(H):
    # print(t)
    action = dict()
    call[H-t-1] += 1
    for alg in ALGOS:
      start = time.process_time()
      action[alg] = ALGORITHMS[alg](P, R, C, H-t+1, state[alg], Ni, GAMMA)
      time_tk[alg][H-t-1] += (time.process_time() - start)
      assert((action[alg]*C).sum() <= BUDGET + 1e-3)

      reward[alg] += (GAMMA**(t-1))*compute_reward(alg, R, state[alg], action[alg])
      state[alg] = next_state(P, state[alg], action[alg])
      assert(state[alg].sum() == N_BEN)

  rewards_mp.append(reward); calls_mp.append(call); time_taken_mp.append(time_tk)
  print(f'{i_run}', end=' ')

### run the loop

In [ ]:
# Select one of the following data sources
if DATA == "ARMMAN":
  P, R, C, Ni = get_ARMMAN_RMAB()
  P = np.abs(P)  # due to rounding error some values were -0 or approx -1e-14, causing issues later
elif DATA == "ADVERSE":
  P, R, C, Ni = get_adverse_RMAB()


# Rewards
rewards = dict()
for alg in ALGOS: rewards[alg] = np.zeros((H_RANGE.size, N_RUNS))

# Time Taken
time_taken = dict()
calls = np.zeros(H_RANGE[-1])
for alg in ALGOS: time_taken[alg] = np.zeros(H_RANGE[-1])

for i_H in range(H_RANGE.size):
  H = H_RANGE[i_H]  # horizon

  # Multiprocessing
  manager = multiprocessing.Manager()
  rewards_mp = manager.list(); calls_mp = manager.list(); time_taken_mp = manager.list()
  jobs = []  # multiprocessing jobs

  for i_run in range(N_RUNS):
    if DATA == "RANDOM":
      P, R, C, Ni = get_random_RMAB()
      P = np.abs(P)

    if i_run % 10 == 0:
      print(f"Running H = {H} and run = {i_run}")

    state = dict()
    if DATA == "ADVERSE":
      random_start_state = np.zeros((N_CLS, N_STATES))
      random_start_state[:,0] = N_BEN/2
      random_start_state[:,2] = N_BEN/2
    else:
      random_start_state = get_random_start_state(Ni)
    for alg in ALGOS: state[alg] = random_start_state  # start at same (random) state

    proc = multiprocessing.Process(target=worker, args=(P, R, C, Ni, state, rewards_mp, calls_mp, time_taken_mp))
    jobs.append(proc)
    proc.start()

  for proc in jobs:
    proc.join()
  
  # combine the collected statistics from the parallel runs
  for i_run in range(N_RUNS):
    calls[:calls_mp[i_run].size] += calls_mp[i_run]
    for alg in ALGOS: 
      rewards[alg][i_H][i_run] = rewards_mp[i_run][alg]
      time_taken[alg][:time_taken_mp[i_run][alg].size] += time_taken_mp[i_run][alg]
  for alg in ALGOS: rewards[alg][i_H] /= (H)

  # save_backup()

In [ ]:
rewards

## plot

In [ ]:
for alg in ALGOS:
  plt.plot(H_RANGE, rewards[alg].mean(axis=-1), label=ALGOS_NAMES[alg])
plt.legend(loc="lower right")
plt.xlabel("Length of Time Horizon")
plt.ylabel("Number of Engaged Beneficiaries / Reward")
plt.title("Average Reward")
plt.show()

In [ ]:
for alg in ALGOS:
  if alg == 'rd':
    continue
  plt.plot(H_RANGE, (rewards[alg] - rewards['rd']).mean(axis=-1), label=ALGOS_NAMES[alg])
plt.legend(loc="lower right")
plt.xlabel("Length of Time Horizon")
plt.ylabel("Reward minus Random's Reward")
plt.title("Reward of Algorithm minus Random's Reward")
plt.show()

In [ ]:
# rewardsDiff = rewards['mf'] - rewards['wi-f']
# plt.plot(H_RANGE, rewardsDiff.mean(axis=-1), 'b')
# plt.fill_between(H_RANGE, 
#                  rewardsDiff.mean(axis=-1) + rewardsDiff.std(axis=-1), 
#                  rewardsDiff.mean(axis=-1) - rewardsDiff.std(axis=-1),
#                  color='c')
# plt.xlabel("Time horizon H")
# plt.ylabel("Difference b/w reward of MF and WI")
# plt.show()

In [ ]:
for alg in ALGOS:
  plt.plot(range(calls.size), time_taken[alg]/calls, label=ALGOS_NAMES[alg])
plt.legend(loc="upper left")
plt.xlabel("Length of Time Horizon")
plt.ylabel("Average Time Taken per Call (sec)")
plt.show()